<img align="right" src="images/tf-small.png" width="128"/>
<img align="right" src="images/etcbc.png"/>
<img align="right" src="images/dans-small.png"/>
<img align="right" src="images/emdros.png"/>


# Search from MQL

Maybe you know MQL.
It is the search template language implemented by Ulrik Sandborg-Petersen in Emdros, and used
by [SHEBANQ](https://shebanq.ancient-data.org).

TF search templates have been inspired by MQL, but they are different.

This notebook shows examples of real-life
[MQL](https://github.com/ETCBC/shebanq/wiki/Documents/MQL-Query-Guide.pdf)
queries on
[SHEBANQ](https://shebanq.ancient-data.org/hebrew/queries). 
and 
expresses them
as Text-Fabric [search templates](https://annotation.github.io/text-fabric/Use/Search/#search-templates).

For more basic examples of TF search, see the
[search](https://github.com/etcbc/bhsa/blob/master/tutorial/search.ipynb)
tutorial.

We get up and running, and then start with the list of examples.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tf.app import use
from tf.core.helpers import project

We want to use a fixed data version, not a changing one, for the purposes of this notebook.

In [3]:
VERSION = 'c'
A = use('bhsa', hoist=globals(), version=VERSION)

	connecting to online GitHub repo annotation/app-bhsa ... connected
Using TF-app in /Users/dirk/text-fabric-data/annotation/app-bhsa/code:
	#d3cf8f0c2ab5d690a0fda14ea31c33da5c5c8483 (latest commit)
	connecting to online GitHub repo etcbc/bhsa ... connected
Using data in /Users/dirk/text-fabric-data/etcbc/bhsa/tf/c:
	rv1.6 (latest release)
	connecting to online GitHub repo etcbc/phono ... connected
Using data in /Users/dirk/text-fabric-data/etcbc/phono/tf/c:
	r1.2 (latest release)
	connecting to online GitHub repo etcbc/parallels ... connected
Using data in /Users/dirk/text-fabric-data/etcbc/parallels/tf/c:
	r1.2 (latest release)


# By Oliver Glanz

## MQL

[Oliver Glanz: PP with adjective followed by noun](https://shebanq.ancient-data.org/hebrew/query?version=4b&id=547)
```
select all objects where
[phrase FOCUS typ = PP
  [word sp= prep]
  [word sp=adjv]
  [word sp=subs]
]
```
64 results having 251 words.

## TF

In [4]:
query = '''
phrase typ=PP
  word sp=prep
  <: word sp=adjv
  <: word sp=subs
'''
results = A.search(query)

  1.54s 64 results


## Comparison

The number of results is right. The number of words that SHEBANQ reports
is the number of words in the phrases of the result. Let us count them:

In [5]:
print(sum([len(L.d(r[0], otype='word')) for r in results]))

251


# By Martijn Naaijer
## MQL

[Martijn Naaijer: Object clauses with >CR](https://shebanq.ancient-data.org/hebrew/query?version=4b&id=997)

```
Select all objects where 

[clause rela = Objc
   [word focus first lex = '>CR']
]
```

157 results

## TF

In [6]:
query = '''
verse
    clause rela=Objc
        =: word lex=>CR
'''
results = A.search(query)
A.table(results, end=10)

  0.51s 96 results


## Comparison

We have fewer cases: 96 instead of 157.
We are working on the ETCBC version 2017, and the query has been executed against 4b.
There have been coding updates that are relevant to this query, e.g. in Genesis 43:27, which is in the results
on SHEBANQ, but not here. In 2017 the `rela` is `Attr`, and not `Objc`:

In [7]:
query = '''
verse book=Genesis chapter=43 verse=27
    clause
        =: word lex=>CR
'''
results = A.search(query)
A.show(results)

  0.51s 1 result


# By Cody Kingham
## MQL

[Cody Kingham: MI Hierarchies. p.18n49. First Person Verbs in Narrative](https://shebanq.ancient-data.org/hebrew/query?version=4b&id=1050)

```
SELECT ALL OBJECTS WHERE

[book
   [clause txt = 'N'
      [word FOCUS sp = verb
        [word ps = p1
         ]
      ]
   ]
]
OR
[book
   [clause txt = '?N'
      [word FOCUS sp = verb
        [word ps = p1
         ]
      ]
   ]
]
```

273 results.

## TF

In [8]:
query = '''
book
    clause txt=N|?N
        word sp=verb ps=p1
'''
results = A.search(query)
A.table(results, end=5)

  0.55s 272 results


## Comparison

One result less. Again, a coding difference between versions. 
Exercise: find out where that happened.

# By Reinoud Oosting
## MQL

[Reinoud Oosting: to go + object marker](https://shebanq.ancient-data.org/hebrew/query?version=4b&id=755)

```
Select all objects
where
 [clause
   [phrase function = Pred OR function = PreC
     [word FOCUS sp = verb AND vs = qal AND lex = "HLK[" ]
         ]
    ..
    [phrase FOCUS
    [word First lex = ">T"]
   ]
 ]
OR
 [clause
    [phrase FOCUS
      [word First lex = ">T" ]
    ]
..
   [phrase function = Pred OR function = PreC
     [word FOCUS sp = verb AND vs = qal AND lex = "HLK["]
   ]
 ]
 ```
 
 4 results.
 

## TF

This is a case where we can simplify greatly because we are not hampered
by automatic constraints on the order of the phrases.

In [9]:
query = '''
clause
  p1:phrase function=Pred|PreC
    word sp=verb vs=qal lex=HLK[
  p2:phrase
    =: word lex=>T
  p1 # p2
'''

results = A.search(query)
A.show(sorted(results), condensed=False)

  1.26s 4 results


# By Reinoud Oosting (ii)
## MQL

[Reinoud Oosting: To establish covenant](https://shebanq.ancient-data.org/hebrew/query?version=4b&id=1485)

```
select all objects
where

 [clause
   [phrase function = Pred OR function = PreC
     [word FOCUS sp = verb AND vs = hif AND lex = "QWM[" ]
         ]
    ..
    [phrase function = Objc
    [word FOCUS lex = "BRJT/" ]
   ]
 ]
OR
 [clause
    [phrase function = Objc
      [word FOCUS lex = "BRJT/" ]
    ]
..
   [phrase function = Pred OR function = PreC
     [word FOCUS sp = verb AND vs = hif AND lex = "QWM["]
   ]
 
]
```

13 results

## TF

In [10]:
query = '''
clause
  phrase function=Pred|PreC
    word sp=verb vs=hif lex=QWM[
  phrase function=Objc
    word lex=BRJT/
'''

results = A.search(query)
resultsx = sorted((L.u(r[0], otype='verse')+r for r in results), key=lambda r: sortKey(r[0]))
A.table(sorted(resultsx))
A.show(resultsx, start=4, end=4)

  1.31s 13 results


# By Reinoud Oosting (iii)
## MQL

[Reinoud Oosting: To find grace in sight of](https://shebanq.ancient-data.org/hebrew/query?version=4b&id=1484)

```
select all objects
where

 [clause
   [phrase FOCUS function = Pred OR function = PreC
     [word sp = verb AND vs = qal AND lex = "MY>[" ]
         ]
    ..
    [phrase function = Objc
    [word FOCUS lex = "XN/" ]
   ]
[phrase function = Cmpl
[word FOCUS lex = "B"]
[word FOCUS lex = "<JN/"]
]
 ]
OR
 [clause
    [phrase function = Objc
      [word FOCUS lex = "XN/" ]
    ]
[phrase function = Cmpl
[word FOCUS lex = "B"]
[word FOCUS lex = "<JN/"]
..
   [phrase function = Pred OR function = PreC
     [word FOCUS sp = verb AND vs = qal AND lex = "MY>["]
   ]
 ]
]

```

38 results

## TF

In [11]:
query = '''
clause
  p1:phrase function=Pred|PreC
    word sp=verb vs=qal lex=MY>[
  p2:phrase function=Objc
    word lex=XN/
  p3:phrase function=Cmpl
    word lex=B
    <: word lex=<JN/
  p2 << p3
'''

results = A.search(query)
resultsx = sorted((L.u(r[0], otype='verse')+r for r in results), key=lambda r: sortKey(r[0]))
A.table(resultsx)

  2.35s 40 results


## Comparison

Two results more. Spot the differences.

# By Stephen Ku
## MQL

[Stephen Ku: Verbless Clauses](https://shebanq.ancient-data.org/hebrew/query?version=4&id=1314)

```
SELECT ALL OBJECTS WHERE 

[clause  
 [phrase function IN (Subj) 
    [phrase_atom NOT rela IN (Appo,Para,Spec)
      [word FOCUS pdp IN (subs,nmpr,prps,prde,prin,adjv)
      ]
    ]
  ]
 NOTEXIST [phrase function IN (Pred)]
 ..
 NOTEXIST [phrase function IN (Pred)]
 [phrase function IN (PreC)
     NOTEXIST [word pdp IN (prep)]
     [word FOCUS pdp IN (subs,nmpr,prin,adjv) AND ls IN (card,ordn)]
 ]
]
```

2303 results with 2129 words in those results.

## TF

We can deal with `NOTEXIST` by means of the quantifier `/without/`.
We can also state that features do *not* have certain values.
And we play with the spatial relations.

In [12]:
query = '''
clause
  phrase function=Subj
  /without/
  <: phrase function=Pred
  /-/
    phrase_atom rela#Appo|Para|Spec
      word pdp=subs|nmpr|prps|prde|prin|adjv
  << phrase function=PreC
  /without/
  :> phrase function=Pred
  /-/
  /without/
    word pdp=prep
  /-/
    word pdp=subs|nmpr|prin|adjv ls=card|ordn
'''

In [13]:
results = A.search(query)
clauses = project(results, 1)
print(f'{len(clauses)} clauses in results')

  1.82s 2288 results
519 clauses in results


## Comparison

We have 15 results less than the MQL query on SHEBANQ.

Let us have a look at some results words and compare them with the result words on SHEBANQ.
It is handy to fetch from SHEBANQ the CSV file with query results.

We have fetched them and stored them in `fromShebanq.csv` in the same directory.
It is a list of words occurring in results, so let's see which clauses are in the SHEBANQ results.

```
book,chapter,verse,monad,text,ktv,phtext,phsep
Genesis,5,4,2169,יְמֵי־,,yᵊmê-,
Genesis,5,4,2170,אָדָ֗ם ,,ʔāḏˈām, 
Genesis,5,4,2175,שְׁמֹנֶ֥ה ,,šᵊmōnˌeh, 
Genesis,5,4,2176,מֵאֹ֖ת ,,mēʔˌōṯ, 
Genesis,5,5,2185,כָּל־,,kol-,
Genesis,5,5,2186,יְמֵ֤י ,,yᵊmˈê, 
Genesis,5,5,2187,אָדָם֙ ,,ʔāḏˌām, 
Genesis,5,5,2190,תְּשַׁ֤ע ,,tᵊšˈaʕ, 
Genesis,5,5,2191,מֵאֹות֙ ,,mēʔôṯ, 
Genesis,5,5,2194,שְׁלֹשִׁ֖ים ,,šᵊlōšˌîm, 
```

In [14]:
shebanqClauses = set()
with open('fromShebanq.csv') as fh:
    for (i, line) in enumerate(fh):
        if i == 0: continue
        fields = line.split(',')
        word = int(fields[3])
        clause = L.u(word, otype='clause')[0]
        shebanqClauses.add(clause)
len(shebanqClauses)

519

That looks good: both methods yield the same amount of clauses.

But we need to be thorough.

In [15]:
clauses == shebanqClauses

False

See? They are not the same clauses.

Let's spot the differences.

In [16]:
tfNotMql = clauses - shebanqClauses
mqlNotTf = shebanqClauses - clauses
print(f'Results of TF  but not MQL: {sorted(tfNotMql)}')
print(f'Results of MQL but not TF : {sorted(mqlNotTf)}')

Results of TF  but not MQL: [482540, 485183, 507066]
Results of MQL but not TF : [427990, 452130, 511777]


## TF yes - MQL no

First we do the results that TF provides, but not MQL.

In [17]:
A.displaySetup(extraFeatures='ls')

In [18]:
newResults = [r for r in results if r[0] in tfNotMql]
newResults

[(482540, 813922, 1076511, 275732, 813923, 275736),
 (482540, 813922, 1076511, 275733, 813923, 275736),
 (485183, 821473, 1084405, 288698, 821474, 288703),
 (485183, 821473, 1084405, 288701, 821474, 288703),
 (507066, 879401, 1143975, 375038, 879402, 375041),
 (507066, 879401, 1143975, 375039, 879402, 375041)]

We are going to inspect them clause by clause.
Note that we have two results per clause, the only difference between the two results is
in column 4, which corresponds to the word in the Subj phrase.

In [19]:
A.show(newResults, condensed=True, withNodes=True)

In all three cases we see a Pred phrase somewhere after the PreC phrase.

The `NOTEXIST` of MQL works a bit subtle: the not-exists claim holds from the place where it appears till the end
of the surrounding context.

So, in fact, the second `NOTEXIST` is redundant. Following the MQL query, the clause cannot have a Pred phrase beyond
the Subj phrase.

## TF no - MQL yes

Before we remedy our TF query to match this effect, let us inspect the clauses delivered by MQL, but not by TF.

Most of the effort in the code below is to furnish appropriate highlighting.

In [20]:
def showClause(clause):
    highlights = {}
    for phrase in L.d(clause, otype='phrase'):
        pf = F.function.v(phrase)
        if pf == 'Subj':
            highlights[phrase] = 'cyan'
            for phraseAtom in L.d(phrase, otype='phrase_atom'):
                rela = F.rela.v(phraseAtom)
                if rela in {'Appo', 'Para', 'Spec'}:
                    continue
                words = L.d(phraseAtom, otype='word')
                for word in words:
                    pdp = F.pdp.v(word)
                    if pdp in {'subs','nmpr','prps','prde','prin','adjv'}:
                        highlights[word] = 'yellow'
        elif pf == 'PreC':
            highlights[phrase] = 'lightskyblue'
            words = L.d(phrase, otype='word')
            if any(F.pdp.v(word) == 'prep' for word in words):
                continue
            for word in words:
                pdp = F.pdp.v(word)
                ls = F.ls.v(word)
                
                if (
                    ls in {'card', 'ord'} 
                    and
                    pdp in {'subs','nmpr','prin','adjv'}
                ):
                    highlights[word] = 'yellow'
        elif pf == 'Pred':
            highlights[phrase] = 'coral'
    A.pretty(clause, withNodes=True, highlights=highlights)

In [21]:
mqlClauses = sorted(mqlNotTf)

We inspect the cases one by one:

In [22]:
showClause(mqlClauses[0])

What could be wrong here? The only violation could be in the *gap*. What happens before the PreC phrase?
If there is an adjacent Pred phrase, it explains why this does not show up in the TF query results.
Let's find out.

In [23]:
xPhrase = L.u(2189, otype='phrase')[0]
A.pretty(xPhrase, withNodes=True, highlights={xPhrase: 'coral' if F.function.v(xPhrase) == 'Pred' else 'lightyellow'})

Clearly, this is the culprit.
it is in the same clause.

In [24]:
showClause(mqlClauses[1])

Again, a gap just before the Prec phrase. Indeed:

In [25]:
xPhrase = L.u(132678, otype='phrase')[0]
A.pretty(xPhrase, withNodes=True, highlights={xPhrase: 'coral' if F.function.v(xPhrase) == 'Pred' else 'lightyellow'})

In [26]:
showClause(mqlClauses[2])

We are getting used to it!

In [27]:
xPhrase = L.u(403004, otype='phrase')[0]
A.pretty(xPhrase, withNodes=True, highlights={xPhrase: 'coral' if F.function.v(xPhrase) == 'Pred' else 'lightyellow'})

But no, here we have a different cause. Probably a Pred phrase right after the Subj phrase.

In [28]:
xPhrase = L.u(402999, otype='phrase')[0]
A.pretty(xPhrase, withNodes=True, highlights={xPhrase: 'coral' if F.function.v(xPhrase) == 'Pred' else 'lightyellow'})

## Remedy

We have seen all the causes why the TF search and the MQL query produced different results.

Now we are going to remedy the TF query, such that it produces the same results as the MQL.

Let us start with what we just saw: when we stipulate the non-existence of a Pred phrase, we only claim that such
a phrase does not occur in the same clause.

Then we remove the second non-existence claim of a Pred phrase, since the MQL query just stipulates that there is
no Pred phrase after the Subj phrase.

But then we can make the quantifier much simpler. Instead of applying it to the Subj phrase,
we apply it to the enclosing clause. That will solve the problem of phrases outside the clause in one go!

In [29]:
query = '''
c:clause
/without/
  phrase function=Subj
  << phrase function=Pred
/-/
  p:phrase function=Subj
    phrase_atom rela#Appo|Para|Spec
      word pdp=subs|nmpr|prps|prde|prin|adjv
  << phrase function=PreC
  /without/
    word pdp=prep
  /-/
    word pdp=subs|nmpr|prin|adjv ls=card|ordn
'''

In [30]:
results = A.search(query)
clauses = project(results, 1)
print(f'{len(clauses)} clauses in results')

clauses == shebanqClauses

  2.00s 2303 results
519 clauses in results


True

And this is in exact agreement with the MQL query.

As a bonus, let's study this query in order to see what the quantifiers are doing.

In [31]:
S.study(query)

   |     0.00s Feature overview: 111 for nodes; 7 for edges; 2 configs; 8 computed
  0.00s Checking search template ...
  0.00s Setting up search space for 6 objects ...
   |     0.00s "Quantifier on "c:clause"
   |      |   /without/
   |      |   c:clause
   |      |     phrase function=Subj
   |      |     << phrase function=Pred
   |      |   /-/
   |      |     0.79s 4987 nodes to exclude
   |     0.79s reduction from 88121 to 83134 nodes
   |     0.00s "Quantifier on "parent:phrase function=PreC"
   |      |   /without/
   |      |   parent:phrase function=PreC
   |      |     word pdp=prep
   |      |   /-/
   |      |     0.76s 6370 nodes to exclude
   |     0.76s reduction from 19395 to 13025 nodes
  1.17s Constraining search space with 6 relations ...
  2.00s 	5 edges thinned
  2.00s Setting up retrieval plan ...
  2.00s Ready to deliver results from 5321 nodes
Iterate over S.fetch() to get the results
See S.showPlan() to interpret the results


# By Dirk Roorda
## MQL

[Dirk Roorda: Yesh](https://shebanq.ancient-data.org/hebrew/query?version=4b&id=556)

```
select all objects where
[book [chapter [verse
[clause
    [clause_atom
        [phrase
            [phrase_atom
                [word focus lex="JC/" OR lex=">JN/"]
            ]
        ]
    ]
]
]]]
```

926 results

## TF

In [32]:
query = '''
verse
  clause
    clause_atom
      phrase
        phrase_atom
          word lex=JC/|>JN/
'''

results = A.search(query)
A.table(sorted(results), end=10)

  0.55s 926 results
